In [2]:
# load modules
## Data processing and DA modules
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, zoomed_inset_axes
## Dealing with big data and netcdf
import xarray as xr
from netCDF4 import Dataset
## ROMS packages
from xgcm import Grid
## color maps
import cmaps
import cmocean
## mapping packages
import cartopy.crs as ccrs
import cartopy.feature as cfeature
## System tools and python configuration
import os
import glob

Firs round of processing - EKE calcs

In [2]:
ds=xr.open_dataset("/seq001/data/roms_his_DailyNudge_2018.nc")
ds

<xarray.Dataset>
Dimensions:         (tracer: 2, boundary: 4, s_rho: 31, s_w: 32, Nuser: 1,
                     eta_rho: 600, xi_rho: 800, eta_u: 600, xi_u: 799,
                     eta_v: 599, xi_v: 800, eta_psi: 599, xi_psi: 799,
                     ocean_time: 365)
Coordinates:
  * s_rho           (s_rho) float64 -0.9839 -0.9516 ... -0.04839 -0.01613
  * s_w             (s_w) float64 -1.0 -0.9677 -0.9355 ... -0.06452 -0.03226 0.0
    x_rho           (eta_rho, xi_rho) float64 ...
    y_rho           (eta_rho, xi_rho) float64 ...
    x_u             (eta_u, xi_u) float64 ...
    y_u             (eta_u, xi_u) float64 ...
    x_v             (eta_v, xi_v) float64 ...
    y_v             (eta_v, xi_v) float64 ...
    x_psi           (eta_psi, xi_psi) float64 ...
    y_psi           (eta_psi, xi_psi) float64 ...
  * ocean_time      (ocean_time) datetime64[ns] 2025-12-28 ... 2026-12-27
Dimensions without coordinates: tracer, boundary, Nuser, eta_rho, xi_rho,
                                eta_u, xi_u, eta_v, xi_v, eta_psi, xi_psi
Data variables: (12/108)
    ntimes          int32 ...
    ndtfast         int32 ...
    dt              float64 ...
    dtfast          float64 ...
    dstart          datetime64[ns] ...
    nHIS            int32 ...
    ...              ...
    lat_u           (eta_u, xi_u) float64 ...
    lat_v           (eta_v, xi_v) float64 ...
    lon_psi         (eta_psi, xi_psi) float64 ...
    lon_rho         (eta_rho, xi_rho) float64 ...
    lon_u           (eta_u, xi_u) float64 ...
    lon_v           (eta_v, xi_v) float64 ...
Attributes: (12/36)
    file:                       roms_his_0020.nc
    format:                     netCDF-3 64bit offset file
    Conventions:                CF-1.4, SGRID-0.3
    type:                       ROMS grid file
    title:                      South-east Queensland, 1/100 (900m) degree re...
    var_info:                   ROMS/External/varinfo.yaml
    ...                         ...
    tiling:                     024x020
    history:                    Mon Mar 18 23:37:01 2024: ncks -A ROMS_grid_c...
    ana_file:                   ROMS/Functionals/ana_btflux.h
    CPP_options:                SEQLD, ANA_BSFLUX, ANA_BTFLUX, ASSUMED_SHAPE,...
    NCO:                        netCDF Operators version 5.0.6 (Homepage = ht...
    history_of_appended_files:  Mon Mar 18 23:37:01 2024: Appended file ROMS_...

In [3]:
ds=ds.isel(xi_rho=slice(0,600))
ds=ds.set_coords(("lon_rho","lat_rho"))
ds = ds[['temp','u_eastward','v_northward','zeta']]
ds

<xarray.Dataset>
Dimensions:      (ocean_time: 365, s_rho: 31, eta_rho: 600, xi_rho: 600)
Coordinates:
  * s_rho        (s_rho) float64 -0.9839 -0.9516 -0.9194 ... -0.04839 -0.01613
    x_rho        (eta_rho, xi_rho) float64 ...
    y_rho        (eta_rho, xi_rho) float64 ...
  * ocean_time   (ocean_time) datetime64[ns] 2025-12-28 ... 2026-12-27
    lat_rho      (eta_rho, xi_rho) float64 ...
    lon_rho      (eta_rho, xi_rho) float64 ...
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    temp         (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    u_eastward   (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    v_northward  (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    zeta         (ocean_time, eta_rho, xi_rho) float32 ...
Attributes: (12/36)
    file:                       roms_his_0020.nc
    format:                     netCDF-3 64bit offset file
    Conventions:                CF-1.4, SGRID-0.3
    type:                       ROMS grid file
    title:                      South-east Queensland, 1/100 (900m) degree re...
    var_info:                   ROMS/External/varinfo.yaml
    ...                         ...
    tiling:                     024x020
    history:                    Mon Mar 18 23:37:01 2024: ncks -A ROMS_grid_c...
    ana_file:                   ROMS/Functionals/ana_btflux.h
    CPP_options:                SEQLD, ANA_BSFLUX, ANA_BTFLUX, ASSUMED_SHAPE,...
    NCO:                        netCDF Operators version 5.0.6 (Homepage = ht...
    history_of_appended_files:  Mon Mar 18 23:37:01 2024: Appended file ROMS_...

In [ ]:
#took about 18 mins
#mem was around 50gb
ds["eke"] = 0.5*((ds.u_eastward - ds.u_eastward.mean("ocean_time"))**2 + (ds.v_northward - ds.v_northward.mean("ocean_time"))**2)

In [4]:
u_bar = ds.u_eastward.mean("ocean_time")
v_bar = ds.v_northward.mean("ocean_time")

print('calc velocity anomalies')
u_prime = ds.u_eastward - u_bar
v_prime = ds.v_northward - v_bar
del(u_bar)
del(v_bar)

print('calc eke')
eke = 0.5*(u_prime**2 + v_prime**2)
del(u_prime)
del(v_prime)

ds["eke"] = eke
del(eke)

calc velocity anomalies
calc eke


In [5]:
ds.to_netcdf("/seq001/data/roms_his_DailyNudge_2018_daily_SSA.nc")

2nd round of processing - z_rho

In [3]:
ds=xr.open_dataset("/seq001/data/roms_his_DailyNudge_2018_daily_SSA.nc")
ds

<xarray.Dataset>
Dimensions:      (ocean_time: 365, s_rho: 31, eta_rho: 600, xi_rho: 600)
Coordinates:
  * s_rho        (s_rho) float64 -0.9839 -0.9516 -0.9194 ... -0.04839 -0.01613
    x_rho        (eta_rho, xi_rho) float64 ...
    y_rho        (eta_rho, xi_rho) float64 ...
  * ocean_time   (ocean_time) datetime64[ns] 2025-12-28 ... 2026-12-27
    lat_rho      (eta_rho, xi_rho) float64 ...
    lon_rho      (eta_rho, xi_rho) float64 ...
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    temp         (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    u_eastward   (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    v_northward  (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    zeta         (ocean_time, eta_rho, xi_rho) float32 ...
    eke          (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
Attributes: (12/36)
    file:                       roms_his_0020.nc
    format:                     netCDF-3 64bit offset file
    Conventions:                CF-1.4, SGRID-0.3
    type:                       ROMS grid file
    title:                      South-east Queensland, 1/100 (900m) degree re...
    var_info:                   ROMS/External/varinfo.yaml
    ...                         ...
    tiling:                     024x020
    history:                    Mon Mar 18 23:37:01 2024: ncks -A ROMS_grid_c...
    ana_file:                   ROMS/Functionals/ana_btflux.h
    CPP_options:                SEQLD, ANA_BSFLUX, ANA_BTFLUX, ASSUMED_SHAPE,...
    NCO:                        netCDF Operators version 5.0.6 (Homepage = ht...
    history_of_appended_files:  Mon Mar 18 23:37:01 2024: Appended file ROMS_...

In [4]:
ds_grid=xr.open_dataset("/seq001/data/roms_his_DailyNudge_2018_5daymn_grid_vars.nc")
ds_grid=ds_grid[['z_rho0','dx']]
ds_grid=ds_grid.isel(xi_rho=slice(0,600))
ds_grid

<xarray.Dataset>
Dimensions:  (eta_rho: 600, xi_rho: 600, s_rho: 31)
Coordinates:
    z_rho0   (eta_rho, xi_rho, s_rho) float64 ...
  * s_rho    (s_rho) float64 -0.9839 -0.9516 -0.9194 ... -0.04839 -0.01613
    lat_rho  (eta_rho, xi_rho) float64 ...
    lon_rho  (eta_rho, xi_rho) float64 ...
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    dx       (eta_rho, xi_rho) float64 ...
Attributes: (12/38)
    CDI:                        Climate Data Interface version 2.0.4 (https:/...
    Conventions:                CF-1.4, SGRID-0.3
    file:                       roms_his_0020.nc
    format:                     netCDF-3 64bit offset file
    type:                       ROMS grid file
    title:                      South-east Queensland, 1/100 (900m) degree re...
    ...                         ...
    history:                    Mon Feb 19 09:22:01 2024: ncks -A ROMS_grid_c...
    ana_file:                   ROMS/Functionals/ana_btflux.h
    CPP_options:                SEQLD, ANA_BSFLUX, ANA_BTFLUX, ASSUMED_SHAPE,...
    CDO:                        Climate Data Operators version 2.0.4 (https:/...
    NCO:                        netCDF Operators version 5.0.6 (Homepage = ht...
    history_of_appended_files:  Mon Feb 19 09:22:01 2024: Appended file ROMS_...

In [5]:
#merge depths to ds
ds=xr.merge([ds,ds_grid])
ds

<xarray.Dataset>
Dimensions:      (ocean_time: 365, s_rho: 31, eta_rho: 600, xi_rho: 600)
Coordinates:
  * s_rho        (s_rho) float64 -0.9839 -0.9516 -0.9194 ... -0.04839 -0.01613
    x_rho        (eta_rho, xi_rho) float64 ...
    y_rho        (eta_rho, xi_rho) float64 ...
  * ocean_time   (ocean_time) datetime64[ns] 2025-12-28 ... 2026-12-27
    lat_rho      (eta_rho, xi_rho) float64 -30.0 -30.0 -30.0 ... -24.01 -24.01
    lon_rho      (eta_rho, xi_rho) float64 151.5 151.5 151.5 ... 157.5 157.5
    z_rho0       (eta_rho, xi_rho, s_rho) float64 ...
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    temp         (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    u_eastward   (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    v_northward  (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    zeta         (ocean_time, eta_rho, xi_rho) float32 ...
    eke          (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    dx           (eta_rho, xi_rho) float64 ...
Attributes: (12/36)
    file:                       roms_his_0020.nc
    format:                     netCDF-3 64bit offset file
    Conventions:                CF-1.4, SGRID-0.3
    type:                       ROMS grid file
    title:                      South-east Queensland, 1/100 (900m) degree re...
    var_info:                   ROMS/External/varinfo.yaml
    ...                         ...
    tiling:                     024x020
    history:                    Mon Mar 18 23:37:01 2024: ncks -A ROMS_grid_c...
    ana_file:                   ROMS/Functionals/ana_btflux.h
    CPP_options:                SEQLD, ANA_BSFLUX, ANA_BTFLUX, ASSUMED_SHAPE,...
    NCO:                        netCDF Operators version 5.0.6 (Homepage = ht...
    history_of_appended_files:  Mon Mar 18 23:37:01 2024: Appended file ROMS_...

In [8]:
ds.to_netcdf("/seq001/data/roms_his_DailyNudge_2018_daily_SSA_processed.nc")